<a href="https://colab.research.google.com/github/riccardocappi/Text-Adversarial-Attack/blob/adversarial-training/adversarial_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Dependencies

In [1]:
from builtins import len
!pip install textattack

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 445.7/445.7 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.3/388.3 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 769.7/769.7 kB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 46.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.3/143.3 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 26.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 29.0 MB/s eta 0:00:00
  Using cached

## Imports

In [11]:
import textattack
import transformers
from textattack.datasets import HuggingFaceDataset
from textattack import Attacker
from textattack.attack_recipes import DeepWordBugGao2018

# Helper methods

In [12]:
class FixedHuggingFaceDataset(HuggingFaceDataset):
    def __init__(self, name_or_dataset, subset=None, split="train", dataset_columns=None, label_map=None,
                 label_names=None, output_scale_factor=None, shuffle=False, seed=69, subset_size=None, offset=0):
        super().__init__(name_or_dataset=name_or_dataset, subset=subset, split=split, dataset_columns=dataset_columns,
                         label_map=label_map, label_names=label_names, output_scale_factor=output_scale_factor,
                         shuffle=shuffle)
        if shuffle:
            self._dataset = self._dataset.shuffle(seed=seed).flatten_indices()
        if subset_size is not None:
            self._dataset = self._dataset.skip(offset).take(subset_size)

# Loading dataset and model
Getting train data in order to find attack transformation to add into the training set

In [13]:
model = transformers.AutoModelForSequenceClassification.from_pretrained("textattack/bert-base-uncased-imdb")
tokenizer = transformers.AutoTokenizer.from_pretrained("textattack/bert-base-uncased-imdb")
model_wrapper = textattack.models.wrappers.HuggingFaceModelWrapper(model, tokenizer)

# Adversarial training DeepWordBugGao2018

In [28]:
attack = DeepWordBugGao2018.build(model_wrapper)
subset_train = FixedHuggingFaceDataset("imdb", split="train", subset_size=1024, shuffle=True)
subset_eval = FixedHuggingFaceDataset("imdb", split="test", subset_size=64, shuffle=True)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Loading datasets dataset imdb, split train.
textattack: Loading datasets dataset imdb, split test.


In [ ]:
attack_args = textattack.AttackArgs(num_examples=10, parallel=True)
attacker = Attacker(attack, subset_eval, attack_args)
adv_exp_bae_bert_imdb = attacker.attack_dataset()

textattack: Running 1 worker(s) on 1 GPU(s).
textattack: Worklist size: 10
textattack: Worklist candidate size: 54
  0%|          | 0/10 [00:00<?, ?it/s]

In [17]:
training_args = textattack.TrainingArgs(
    num_epochs=5,
    num_clean_epochs=0,
    num_train_adv_examples=128,
    learning_rate=5e-5,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    attack_epoch_interval=2,
    log_to_tb=True,
    parallel=True
)

trainer = textattack.Trainer(
    model_wrapper,
    "classification",
    attack,
    subset_train,
    subset_eval,
    training_args
)

## Train

In [18]:
trainer.train()

textattack: Writing logs to ./outputs/2024-05-13-14-30-48-967309/train_log.txt.
textattack: Wrote original training args to ./outputs/2024-05-13-14-30-48-967309/training_args.json.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
textattack: ***** Running training *****
textattack:   Num examples = 100
textattack:   Num epochs = 3
textattack:   Num clean epochs = 0
textattack:   Instantaneous batch size per device = 8
textattack:   Total train batch size (w. parallel, distributed & accumulation) = 8
textattack:   Gradient accumulation steps = 1
textattack:   Total optimization steps = 42
textattack: ==========================================================
textattack: Epoch 1
textattack: Attacking model to generate new adversaria

Loss 0.07595: 100%|██████████| 14/14 [00:11<00:00,  1.21it/s]
textattack: Train accuracy: 97.27%
textattack: Eval accuracy: 90.00%
textattack: Best score found. Saved model to ./outputs/2024-05-13-14-30-48-967309/best_model/
textattack: ==========================================================
textattack: Epoch 2
textattack: Attacking model to generate new adversarial training set...
[Succeeded / Failed / Skipped / Total] 10 / 13 / 0 / 23: 100%|██████████| 10/10 [06:55<00:00, 41.52s/it]
textattack: Total number of attack results: 23
textattack: Attack success rate: 43.48% [10 / 23]


Loss 0.06149: 100%|██████████| 14/14 [00:10<00:00,  1.31it/s]
textattack: Train accuracy: 98.18%
textattack: Eval accuracy: 90.00%
textattack: ==========================================================
textattack: Epoch 3
textattack: Attacking model to generate new adversarial training set...
[Succeeded / Failed / Skipped / Total] 10 / 25 / 0 / 35: 100%|██████████| 10/10 [10:15<00:00, 61.59s/it]
textattack: Total number of attack results: 35
textattack: Attack success rate: 28.57% [10 / 35]


Loss 0.04937: 100%|██████████| 14/14 [00:10<00:00,  1.32it/s]
textattack: Train accuracy: 99.09%
textattack: Eval accuracy: 90.00%
textattack: Wrote README to ./outputs/2024-05-13-14-30-48-967309/README.md.


In [27]:
model_wrapper_fine_tuned = trainer.model_wrapper
fine_tuned_attack = DeepWordBugGao2018.build(model_wrapper_fine_tuned)
fine_tuned_attacker = Attacker(fine_tuned_attack, subset_eval, attack_args)
fine_tuned_eval = fine_tuned_attacker.attack_dataset()

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Running 1 worker(s) on 1 GPU(s).
textattack: Worklist size: 50
textattack: Worklist candidate size: 50
[Succeeded / Failed / Skipped / Total] 0 / 1 / 0 / 1:   2%|▏         | 1/50 [00:24<19:55, 24.40s/it]

--------------------------------------------- Result 1 ---------------------------------------------
[[Negative (100%)]] --> [[[FAILED]]]

I love sci-fi and am willing to put up with a lot. Sci-fi movies/TV are usually underfunded, under-appreciated and misunderstood. I tried to like this, I really did, but it is to good TV sci-fi as Babylon 5 is to Star Trek (the original). Silly prosthetics, cheap cardboard sets, stilted dialogues, CG that doesn't match the background, and painfully one-dimensional characters cannot be overcome with a 'sci-fi' setting. (I'm sure there are those of you out there who think Babylon 5 is good sci-fi TV. It's not. It's clichéd and uninspiring.) While US viewers might like emotion and character development, sci-fi is a genre that does not take itself seriously (cf. Star Trek). It may treat important issues, yet not as a serious philosophy. It's really difficult to care about the characters here as they are not simply foolish, just missing a spark of life. 

[Succeeded / Failed / Skipped / Total] 1 / 1 / 0 / 2:   4%|▍         | 2/50 [00:35<14:12, 17.75s/it]

--------------------------------------------- Result 2 ---------------------------------------------
[[Negative (100%)]] --> [[Positive (55%)]]

Worth the entertainment value of a rental, especially if you like action movies. This one features the usual [[car]] chases, fights with the great Van Damme kick style, shooting battles with the 40 shell load shotgun, and even terrorist style bombs. All of this is entertaining and competently [[handled]] but there is nothing that really blows you away if you've seen your share before.<br /><br />The plot is made interesting by the inclusion of a rabbit, which is clever but [[hardly]] profound. Many of the characters are heavily stereotyped -- the angry veterans, the terrified illegal aliens, the crooked cops, the indifferent feds, the bitchy tough lady station head, the crooked politician, the fat federale who looks like he was typecast as the Mexican in a Hollywood movie from the 1940s. All passably acted but again [[nothing]] [[special]].<br

[Succeeded / Failed / Skipped / Total] 2 / 1 / 0 / 3:   6%|▌         | 3/50 [00:39<10:14, 13.07s/it]

...

--------------------------------------------- Result 49 ---------------------------------------------
[[Negative (100%)]] --> [[[FAILED]]]

No, no, no, no, no, no, NO! This is not a film, this is an excuse to show people dancing. This is just not good. Even the dancing is slow and not half as entertaining as the mediocre 'Dirty Dancing', let alone any other good dance movie.<br /><br />Is it a love story? Is it a musical? Is it a drama? Is it a comedy? It's not that this movie is a bit of all, it's that this movie fails at everything it attempts to be. The film turns out to be even more meaningless as the film progresses.<br /><br />Acting is terrible from all sides, the screenplay is definitely trying to tell us something about relationship but fails miserably.<br /><br />WATCH FOR THE MOMENT - When Patrick Stewart enters the scene and you think the film might get better as he brightens up the dull atmosphere. For a second.


[Succeeded / Failed / Skipped / Total] 14 / 33 / 3 / 50: 100%|██████████| 50/50 [11:47<00:00, 14.15s/it]

--------------------------------------------- Result 50 ---------------------------------------------
[[Negative (100%)]] --> [[Positive (55%)]]

I found this movie [[really]] [[hard]] to [[sit]] through, my attention kept wandering off the tv. As far as [[romantic]] movies go..this one is the [[worst]] I've [[seen]]. Don't [[bother]] with it.

I found this movie [[reall]] [[hrad]] to [[isit]] through, my attention kept wandering off the tv. As far as [[romankic]] movies go..this one is the [[worts]] I've [[sene]]. Don't [[Fother]] with it.


[Succeeded / Failed / Skipped / Total] 14 / 33 / 3 / 50: 100%|██████████| 50/50 [11:49<00:00, 14.19s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 14     |
| Number of failed attacks:     | 33     |
| Number of skipped attacks:    | 3      |
| Original accuracy:            | 94.0%  |
| Accuracy under attack:        | 66.0%  |
| Attack success rate:          | 29.79% |
| Average perturbed word %:     | 7.07%  |
| Average num. words per input: | 204.38 |
| Avg num queries:              | 287.62 |
+-------------------------------+--------+
